In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from functools import reduce
from decode import read_and_decode

In [ ]:
df = pd.read_csv('./train.csv')

In [ ]:
#Global Variable Setup
train_whole_sample_size = 9996
test_whole_sample_size = 9996
gesture_class = len(df.landmark_id.unique())
train_batch_size = 32
test_batch_size = 32
train_path = "./Landmark_train.tfrecords"
graph_path = "./tensorboard"
cnn_model_save_path = "./cnn_model/cnn_model.ckpt"

In [ ]:
#training set
img_train_batch, labels_train_batch = read_and_decode(train_path, train_batch_size, train_whole_sample_size)
train_label = tf.one_hot(labels_train_batch, gesture_class, 1, 0)


#test set
img_test_batch, labels_test_batch = read_and_decode(train_path, test_batch_size, test_whole_sample_size)
test_label = tf.one_hot(labels_test_batch, gesture_class, 1, 0)


### create session
sess = tf.InteractiveSession()

### create graph placement
x = tf.placeholder(tf.float32, shape=[None, 224, 224, 3], name="images")
y = tf.placeholder(tf.float32, shape=[None, gesture_class,], name="labels")

In [ ]:
### Layer Setup Function
def weight_variable(shape, f_name):
    initial = tf.truncated_normal(shape, mean=0, stddev=0.1)
    return tf.Variable(initial, name=f_name)


def bias_variable(shape, f_name):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial, name=f_name)


def Conv2d_Filter(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding="SAME")


def max_pooling_2x2(x, name):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding="SAME", name=name)

In [ ]:
def get_weight_bias(name):
    if data_dict is not None:
        weight_name = "_weight"
        bias_name = "_bias"
        weight = tf.Variable(data_dict[name][0], name = name + weight_name)
        bias =  tf.Variable(data_dict[name][1], name = name + bias_name)
    return weight, bias

## Transfer Learning: Pre-trained VGG19

In [ ]:
data_dict = np.load("./vgg19.npy", encoding='latin1').item()

### 1.1)conv1_1: conv 3x3 64 (RELU)
- Input: 224x224x3
- Type: Conv
- size: 3x3
- channel: 64
- strides: 1
- Output: 224x224x64


In [ ]:
with tf.name_scope('conv1_1'):
    #W_conv1 = weight_variable([3,3,3,64], 'W_conv1')
    #b_conv1 = bias_variable([64], 'b_conv1')
    W_conv1_1, b_conv1_1 = get_weight_bias('conv1_1')
    with tf.name_scope('h_conv1'):
        h_conv1_1 = tf.nn.relu(Conv2d_Filter(x, W_conv1_1) + b_conv1_1)

### 1.2)conv1_2: conv 3x3 64 (RELU)

- Input: 224x224x64
- Type: Conv
- size: 3x3
- channel: 64
- strides: 1
- Output: 224x224x64

In [ ]:
with tf.name_scope('conv1_2'):
    W_conv1_2, b_conv1_2 = get_weight_bias('conv1_2')
    with tf.name_scope('h_conv2'):
        h_conv1_2 = tf.nn.relu(Conv2d_Filter(h_conv1_1, W_conv1_2) + b_conv1_2)

### 1.3)pool1: max pooling 2x2 2
- Input: 224x224x64
- pool: 2x2
- output: 112x112x64


In [ ]:
with tf.name_scope('Pool1'):
    h_pool1 = max_pooling_2x2(h_conv1_2, 'pool1')

 ### 2.1)conv2_1: conv 3x3 128 (RELU)
- Input: 112x112x64
- Type: Conv
- size: 3x3
- channel: 128
- strides: 1
- Output: 112x112x128

In [ ]:
with tf.name_scope('conv2_1'):
    W_conv2_1, b_conv2_1 = get_weight_bias('conv2_1')
    with tf.name_scope('h_conv2'):
        h_conv2_1 = tf.nn.relu(Conv2d_Filter(h_pool1, W_conv2_1) + b_conv2_1)

 ### 2.1)conv2_2: conv 3x3 128 (RELU)
- Input: 112x112x128
- Type: Conv
- size: 3x3
- channel: 128
- strides: 1
- Output: 112x112x128

In [ ]:
with tf.name_scope('conv2_2'):
    W_conv2_2, b_conv2_2 = get_weight_bias('conv2_2')
    with tf.name_scope('h_conv2'):
        h_conv2_2 = tf.nn.relu(Conv2d_Filter(h_conv2_1, W_conv2_2) + b_conv2_2)

### 2.3)pool2: max pooling 2x2 2
- Input: 112x112x128
- pool: 2x2
- output: 56x56x128

In [ ]:
with tf.name_scope('Pool2'):
    h_pool2 = max_pooling_2x2(h_conv2_2, 'pool2')

### 3.1)conv3_1: conv 3x3 256 (RELU)
- Input: 56x56x128
- Type: Conv
- size: 3x3
- channel: 256
- strides: 1
- Output: 56x56x256

In [ ]:
with tf.name_scope('conv3_1'):
    W_conv3_1, b_conv3_1 = get_weight_bias('conv3_1')
    with tf.name_scope('h_conv3_1'):
        h_conv3_1 = tf.nn.relu(Conv2d_Filter(h_pool2, W_conv3_1) + b_conv3_1)

### 3.2)conv3_2: conv 3x3 256 (RELU)
- Input: 56x56x256
- Type: Conv
- size: 3x3
- channel: 256
- strides: 1
- Output: 56x56x256

In [ ]:
with tf.name_scope('conv3_2'):
    W_conv3_2, b_conv3_2 = get_weight_bias('conv3_2')
    with tf.name_scope('h_conv3_2'):
        h_conv3_2 = tf.nn.relu(Conv2d_Filter(h_conv3_1, W_conv3_2) + b_conv3_2)

### 3.3)conv3_3: conv 3x3 256 (RELU)
- Input: 56x56x256
- Type: Conv
- size: 3x3
- channel: 256
- strides: 1
- Output: 56x56x256

In [ ]:
with tf.name_scope('conv3_3'):
    W_conv3_3, b_conv3_3 = get_weight_bias('conv3_3')
    with tf.name_scope('h_conv3_3'):
        h_conv3_3 = tf.nn.relu(Conv2d_Filter(h_conv3_2, W_conv3_3) + b_conv3_3)

### 3.4)conv3_4: conv 3x3 256 (RELU)
- Input: 56x56x256
- Type: Conv
- size: 3x3
- channel: 256
- strides: 1
- Output: 56x56x256

In [ ]:
with tf.name_scope('conv3_4'):
    W_conv3_4, b_conv3_4 = get_weight_bias('conv3_4')
    with tf.name_scope('h_conv3_4'):
        h_conv3_4 = tf.nn.relu(Conv2d_Filter(h_conv3_3, W_conv3_4) + b_conv3_4)

### 3.5)pool3: max pooling 2x2 2
"""
- Input: 56x56x256
- pool: 2x2
- output: 28x28x256
"""

In [ ]:
with tf.name_scope('Pool3'):
    h_pool3 = max_pooling_2x2(h_conv3_4, 'pool3')

### 4.1)conv4_1: conv 3x3 512 (RELU)
- Input: 28x28x256
- Type: Conv
- size: 3x3
- channel: 512
- strides: 1
- Output: 28x28x512

In [ ]:
with tf.name_scope('conv4_1'):
    W_conv4_1, b_conv4_1 = get_weight_bias('conv4_1')
    with tf.name_scope('h_conv4_1'):
        h_conv4_1 = tf.nn.relu(Conv2d_Filter(h_pool3, W_conv4_1) + b_conv4_1)

In [ ]:
with tf.name_scope('conv4_2'):
    W_conv4_2, b_conv4_2 = get_weight_bias('conv4_2')
    with tf.name_scope('h_conv4_2'):
        h_conv4_2 = tf.nn.relu(Conv2d_Filter(h_conv4_1, W_conv4_2) + b_conv4_2)

In [ ]:
with tf.name_scope('conv4_3'):
    W_conv4_3, b_conv4_3 = get_weight_bias('conv4_3')
    with tf.name_scope('h_conv4_3'):
        h_conv4_3 = tf.nn.relu(Conv2d_Filter(h_conv4_2, W_conv4_3) + b_conv4_3)

In [ ]:
with tf.name_scope('conv4_4'):
    W_conv4_4, b_conv4_4 = get_weight_bias('conv4_4')
    with tf.name_scope('h_conv4_4'):
        h_conv4_4 = tf.nn.relu(Conv2d_Filter(h_conv4_3, W_conv4_4) + b_conv4_4)

In [ ]:
with tf.name_scope('Pool4'):
    h_pool4 = max_pooling_2x2(h_conv4_4, 'pool4')

In [ ]:
with tf.name_scope('conv5_1'):
    W_conv5_1, b_conv5_1 = get_weight_bias('conv5_1')
    with tf.name_scope('h_conv5_1'):
        h_conv5_1 = tf.nn.relu(Conv2d_Filter(h_pool4, W_conv5_1) + b_conv5_1)

In [ ]:
with tf.name_scope('conv5_2'):
    W_conv5_2, b_conv5_2 = get_weight_bias('conv5_2')
    with tf.name_scope('h_conv5_2'):
        h_conv5_2 = tf.nn.relu(Conv2d_Filter(h_conv5_1, W_conv5_2) + b_conv5_2)

In [ ]:
with tf.name_scope('conv5_3'):
    W_conv5_3, b_conv5_3 = get_weight_bias('conv5_3')
    with tf.name_scope('h_conv5_3'):
        h_conv5_3 = tf.nn.relu(Conv2d_Filter(h_conv5_2, W_conv5_3) + b_conv5_3)

In [ ]:
with tf.name_scope('conv5_4'):
    W_conv5_4, b_conv5_4 = get_weight_bias('conv5_4')
    with tf.name_scope('h_conv5_4'):
        h_conv5_4 = tf.nn.relu(Conv2d_Filter(h_conv5_3, W_conv5_4) + b_conv5_4)

In [ ]:
with tf.name_scope('Pool5'):
    h_pool5 = max_pooling_2x2(h_conv5_4, 'pool5')

In [ ]:
with tf.name_scope("fc6"):
    W_fc6, b_fc6 = get_weight_bias('fc6')
    
    with tf.name_scope('Pool5_flat'):
        h_pool6_flat = tf.reshape(h_pool5, [-1, 25088])

    with tf.name_scope('h_fc1'):
        h_fc6 = tf.nn.relu(tf.matmul(h_pool6_flat, W_fc6) + b_fc6)

    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    h_fc6_drop = tf.nn.dropout(h_fc6, keep_prob, name="h_fc6_drop")

In [ ]:
with tf.name_scope("fc7"):
    W_fc7, b_fc7 = get_weight_bias('fc7')
    
    with tf.name_scope('Pool5_flat'):
        h_pool7_flat = tf.reshape(h_fc6_drop, [-1, 4096])

    with tf.name_scope('h_fc1'):
        h_fc7 = tf.nn.relu(tf.matmul(h_pool7_flat, W_fc7) + b_fc7)

    keep_prob = tf.placeholder(tf.float32, name="keep_prob")
    h_fc7_drop = tf.nn.dropout(h_fc7, keep_prob, name="h_fc6_drop")

In [ ]:
###  13)Layer 8: Readout Layer (Softmax Layer)
with tf.name_scope('fc8'):
    W_fc8 = weight_variable([4096, gesture_class], 'fc8')
    b_fc8 = bias_variable([gesture_class], 'fc8')
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(tf.matmul(h_fc7_drop, W_fc8) + b_fc8, name="my_prediction")

# Define functions and train the model

In [ ]:
with tf.name_scope('Corss_Entropy'):
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction),
                                   name="loss")
    tf.summary.scalar('corss_entropy', cross_entropy)

with tf.name_scope('Train_step'):
    train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy, name="train_step")

correct_prediction = tf.equal(tf.arg_max(prediction, 1), tf.arg_max(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
tf.summary.scalar('accuracy', accuracy)
merged = tf.summary.merge_all()

###Start to Train
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    train_writer = tf.summary.FileWriter(graph_path, sess.graph);

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord, sess=sess)

    saver = tf.train.Saver()
    max_acc = 0

    for i in range(1001):

        img_xs, label_xs = sess.run([img_train_batch, train_label])
        sess.run(train_step, feed_dict={x: img_xs, y: label_xs, keep_prob: 0.75})

        if (i % 10) == 0:
            print("The", i, "Train")
            img_test_xs, label_test_xs = sess.run([img_test_batch, test_label])
            acc = sess.run(accuracy, feed_dict={x: img_test_xs, y: label_test_xs, keep_prob: 1.0})
            print("Itsers = " + str(i) + "  Accuracy: " + str(acc))

            summay = sess.run(merged, feed_dict={x: img_test_xs, y: label_test_xs, keep_prob: 1.0})

            train_writer.add_summary(summay, i)

            if max_acc < acc:
                max_acc = acc
                saver.save(sess, save_path=cnn_model_save_path)

            if acc > 0.50:
                break

    train_writer.close()

    coord.request_stop()
    coord.join(threads)
    saver.save(sess, save_path=cnn_model_save_path)
    sess.close()